<a href="https://colab.research.google.com/github/deburky/boosting-scorecards/blob/main/other_notebooks/fastwoe-gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FastWoe

This notebook tests proportion target rate and GPU acceleration with FAISS.

In [1]:
!pip install fastwoe[faiss]

In [2]:
import fastwoe
print(fastwoe.__version__)

0.1.2.post4


In [3]:
import numpy as np
from scipy.special import expit as sigmoid
import pandas as pd
from fastwoe import FastWoe

def make_synthetic(n_rows=30000, n_features=15, trials_mean=10, seed=7):
    """
    Create aggregated-binomial data:
      X  (n_rows, n_features)
      y_rate = successes / trials   (float in [0,1])
      trials  (>=1 per row)
      p_true  underlying oracle probability used to generate outcomes
    """
    rng = np.random.default_rng(seed)
    X = rng.normal(size=(n_rows, n_features)).astype(np.float32)

    beta = rng.normal(size=(n_features,)).astype(np.float32)
    logit = X @ beta + rng.normal(scale=0.6, size=n_rows).astype(np.float32)
    p = sigmoid(logit)

    trials = rng.poisson(lam=trials_mean, size=n_rows).astype(np.int32) + 1
    successes = rng.binomial(n=trials, p=p).astype(np.int32)
    y_rate = successes / trials

    return X, y_rate.astype(np.float32), trials.astype(np.float32), p.astype(np.float32)

X, y_rate, trials, p_true = make_synthetic()
df = pd.DataFrame(X, columns=[f"x{i}" for i in range(X.shape[1])])
df['y_rate'] = y_rate
df['trials'] = trials
df['p_true'] = p_true

display(df.head())

features = df.columns[:-3]

fw = FastWoe(binning_method='tree', tree_kwargs={"max_depth": 5})
fw.fit(df[features], df['p_true'])

fw.get_binning_summary()
fw.get_mapping("x0").head(10)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,y_rate,trials,p_true
0,0.001230,0.298746,-0.274138,-0.890592,-0.454671,-0.991647,0.060144,1.340215,-0.492207,-0.620475,0.489842,0.356887,0.105414,-0.930468,-0.029252,1.000000,13.0,0.972804
1,0.695303,-1.344215,-0.457616,-1.901223,-1.289538,-1.841735,-0.235091,-1.267447,0.271264,0.156751,-0.186931,-2.516760,-0.538693,-0.048501,0.113309,0.000000,11.0,0.056850
2,-1.530136,-0.477753,-0.978519,-0.808837,1.060899,-0.807535,-0.032522,0.884390,-0.583600,-0.111702,0.110464,0.063782,-1.225056,0.076140,1.358823,1.000000,15.0,0.990801
3,-1.547145,0.859383,0.119354,-0.641470,2.000417,0.762260,-1.199289,0.074516,0.576690,-0.188782,0.682910,-0.066517,0.667248,1.438523,-0.675662,1.000000,10.0,0.997914
4,0.203139,-0.463308,0.127268,-1.187195,-0.579302,-0.196196,0.898764,1.145222,-1.323528,-0.794642,0.646903,-1.992420,-0.463170,-0.097287,1.257015,0.923077,13.0,0.761909


/tmp/ipython-input-64893625.py:38: UserWarning: Detected numerical features: ['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14']. Applying decision tree-based binning with max_depth=5. Use get_binning_summary() to view details.
  fw.fit(df[features], df['p_true'])


,category,count,count_pct,good_count,bad_count,event_rate,woe,woe_se,woe_ci_lower,woe_ci_upper
0,"(-∞, -2.9]",500.0,1.666667,119.0,381.0,0.761046,1.157189,0.105014,0.951365,1.363014
1,"(-2.9, -2.1]",848.0,2.826667,251.0,597.0,0.703735,0.863914,0.075227,0.716472,1.011357
2,"(-2.1, -1.7]",486.0,1.620000,158.0,328.0,0.675088,0.730056,0.096840,0.540254,0.919858
3,"(-1.7, -1.5]",254.0,0.846667,84.0,170.0,0.667342,0.694956,0.133368,0.433558,0.956353
4,"(-1.5, -1.5]",2431.0,8.103333,918.0,1513.0,0.622196,0.497647,0.041836,0.415650,0.579644
5,"(-1.5, -1.0]",17.0,0.056667,3.0,14.0,0.809344,1.444523,0.636209,0.197576,2.691470
6,"(-1.0, -1.0]",856.0,2.853333,344.0,512.0,0.598322,0.397247,0.069714,0.260610,0.533885
7,"(-1.0, -1.0]",856.0,2.853333,344.0,512.0,0.598322,0.397247,0.069714,0.260610,0.533885
8,"(-1.0, -1.0]",856.0,2.853333,344.0,512.0,0.598322,0.397247,0.069714,0.260610,0.533885
9,"(-1.0, -1.0]",856.0,2.853333,344.0,512.0,0.598322,0.397247,0.069714,0.260610,0.533885


## GPU example

In [5]:
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 21.4 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
from fastwoe import FastWoe
import time

# Create dataset
np.random.seed(42)
X = pd.DataFrame({
    'feature1': np.random.normal(0, 1, 10000),
    'feature2': np.random.normal(2, 1, 10000),
    'feature3': np.random.normal(-1, 0.5, 10000)
})
y = np.random.binomial(1, 0.3, 10000)

print("=" * 80)
print("FAISS KMeans CPU vs GPU Performance Comparison")
print("=" * 80)
print(f"Data shape: {X.shape}")
print(f"Target distribution: {pd.Series(y).value_counts().to_dict()}\n")

def run_woe(gpu=False):
    label = "GPU" if gpu else "CPU"
    print(f"Testing FAISS KMeans with {label}{' acceleration' if gpu else '-only'}...")
    print("-" * 50)

    start = time.time()
    woe = FastWoe(
        binning_method='faiss_kmeans',
        faiss_kwargs={'k': 8, 'niter': 50, 'verbose': True, 'gpu': gpu},
        numerical_threshold=10,
        warn_on_numerical=False,
    )
    woe.fit(X, y)
    fit_time = time.time() - start

    start = time.time()
    X_trans = woe.transform(X)
    transform_time = time.time() - start

    print(f"\n✓ {label} FAISS KMeans completed!")
    print(f"{label} Fit time: {fit_time:.4f}s")
    print(f"{label} Transform time: {transform_time:.4f}s")
    print(f"{label} Total time: {fit_time + transform_time:.4f}s")
    print(f"\n{label} Binning summary:\n", woe.get_binning_summary())
    print(f"\n{label} Feature stats:\n", woe.get_feature_stats()[['feature', 'gini', 'iv']])
    print("\n" + "=" * 80)

    return woe, fit_time, transform_time, X_trans

# Run for CPU and GPU
woe_cpu, cpu_fit, cpu_trans, _ = run_woe(gpu=False)
woe_gpu, gpu_fit, gpu_trans, _ = run_woe(gpu=True)

# Performance comparison
print("PERFORMANCE COMPARISON")
print("=" * 80)
print(f"{'Metric':<20} {'CPU (s)':<12} {'GPU (s)':<12} {'Speedup':<12}")
print("-" * 80)
print(f"{'Fit Time':<20} {cpu_fit:<12.4f} {gpu_fit:<12.4f} {cpu_fit/gpu_fit:<12.2f}x")
print(f"{'Transform Time':<20} {cpu_trans:<12.4f} {gpu_trans:<12.4f} {cpu_trans/gpu_trans:<12.2f}x")
print(f"{'Total Time':<20} {cpu_fit + cpu_trans:<12.4f} {gpu_fit + gpu_trans:<12.4f} {(cpu_fit + cpu_trans)/(gpu_fit + gpu_trans):<12.2f}x")

# Quality comparison
print("\n" + "=" * 80)
print("QUALITY COMPARISON")
print("=" * 80)
gini_cpu = woe_cpu.get_feature_stats()['gini'].values
gini_gpu = woe_gpu.get_feature_stats()['gini'].values
print("CPU Gini scores:", gini_cpu)
print("GPU Gini scores:", gini_gpu)
print("Gini difference:", np.abs(gini_cpu - gini_gpu))

FAISS KMeans CPU vs GPU Performance Comparison
Data shape: (10000, 3)
Target distribution: {0: 7004, 1: 2996}

Testing FAISS KMeans with CPU-only...
--------------------------------------------------

✓ CPU FAISS KMeans completed!
CPU Fit time: 0.0854s
CPU Transform time: 0.0309s
CPU Total time: 0.1163s

CPU Binning summary:
     feature  values  n_bins  missing        method
0  feature1   10000       8        0  faiss_kmeans
1  feature2   10000       8        0  faiss_kmeans
2  feature3   10000       8        0  faiss_kmeans

CPU Feature stats:
     feature      gini        iv
0  feature1  0.021769  0.001834
1  feature2  0.033203  0.003654
2  feature3  0.025423  0.002197

Testing FAISS KMeans with GPU acceleration...
--------------------------------------------------


/tmp/ipython-input-4119155634.py:33: UserWarning: Input y is a numpy array. Converting to pandas Series. For better control, convert to Series before passing to fit().
  woe.fit(X, y)
/tmp/ipython-input-4119155634.py:33: UserWarning: Input y is a numpy array. Converting to pandas Series. For better control, convert to Series before passing to fit().
  woe.fit(X, y)



✓ GPU FAISS KMeans completed!
GPU Fit time: 0.1150s
GPU Transform time: 0.0308s
GPU Total time: 0.1458s

GPU Binning summary:
     feature  values  n_bins  missing        method
0  feature1   10000       8        0  faiss_kmeans
1  feature2   10000       8        0  faiss_kmeans
2  feature3   10000       8        0  faiss_kmeans

GPU Feature stats:
     feature      gini        iv
0  feature1  0.021769  0.001834
1  feature2  0.033203  0.003654
2  feature3  0.025423  0.002197

PERFORMANCE COMPARISON
Metric               CPU (s)      GPU (s)      Speedup     
--------------------------------------------------------------------------------
Fit Time             0.0854       0.1150       0.74        x
Transform Time       0.0309       0.0308       1.01        x
Total Time           0.1163       0.1458       0.80        x

QUALITY COMPARISON
CPU Gini scores: [0.02176851 0.03320261 0.02542263]
GPU Gini scores: [0.02176851 0.03320261 0.02542263]
Gini difference: [0. 0. 0.]
